I first started by utilizing the =GOOGLEFINANCE command on Google Sheets to compile basic company information (i.e Market Cap, Profit Margin) based on their tickers, as well as using a public SaaS company database I found. Here is the comprehensive spreadsheet of these values:
https://docs.google.com/spreadsheets/d/1eJjBRiNbxmXz46NK0kBK_lSnL67r07sgCjXFRDalDFI/edit?usp=sharing 

However, this information isn't enough to value a business, particularly a SaaS firm! I wanted to compile valuation metrics from the internet, which could explain debt levels, business value, sales and income insights, as well as cash flows. I utilized the yfinance API to help scrape information for each company from the Yahoo Finance website. 

In [ ]:
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import time
from datetime import datetime
import numpy as np
import requests

def safe_numeric_conversion(value):
    if isinstance(value, (int, float)):
        return value
    elif isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return None
    else:
        return None


# Function to fetch financial data using yfinance
#def get_financial_data(ticker):

    stock = yf.Ticker(ticker)
    financials = stock.financials
    return financials

def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    # Convert numeric columns to float
    for col in financials.columns:
        financials[col] = financials[col].apply(safe_numeric_conversion)
    return financials

# Function to fetch cash flow data using yfinance
#def get_cash_flow_data(ticker):
    stock = yf.Ticker(ticker)
    cash_flow = stock.cashflow
    return cash_flow

def get_cash_flow_data(ticker):
    stock = yf.Ticker(ticker)
    cash_flow = stock.cashflow
    # Convert numeric columns to float
    for col in cash_flow.columns:
        cash_flow[col] = cash_flow[col].apply(safe_numeric_conversion)
    return cash_flow

# Function to fetch income statement data using yfinance
#def get_income_statement_data(ticker):
    stock = yf.Ticker(ticker)
    income_statement = stock.financials
    return income_statement

def get_income_statement_data(ticker):
    stock = yf.Ticker(ticker)
    income_statement = stock.financials
    # Convert numeric columns to float
    for col in income_statement.columns:
        income_statement[col] = income_statement[col].apply(safe_numeric_conversion)
    return income_statement

#fetch balance sheet data
#def get_balance_sheet_data(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    return balance_sheet

def get_balance_sheet_data(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    # Convert numeric columns to float
    for col in balance_sheet.columns:
        balance_sheet[col] = balance_sheet[col].apply(safe_numeric_conversion)
    return balance_sheet

# Function to calculate Gross Margin
#def calculate_gross_margin(income_statement):
    revenue = income_statement.loc['Total Revenue'].iloc[0]
    cogs = income_statement.loc['Cost Of Revenue'].iloc[0]
    gross_margin = (revenue - cogs) / revenue * 100
    return gross_margin

def calculate_gross_margin(income_statement):
    try:
        revenue = income_statement.loc['Total Revenue'].iloc[0]
        cogs = income_statement.loc['Cost Of Revenue'].iloc[0]
        if revenue is None or cogs is None:
            return 'N/A'
        gross_margin = (revenue - cogs) / revenue * 100
        return gross_margin
    except Exception as e:
        print(f"Error calculating gross margin: {e}")
        return 'N/A'

#def get_revenue_growth_rate(ticker_symbol):
    try:
        # Initialize the ticker
        ticker = yf.Ticker(ticker_symbol)
        
        # Fetch income statement data (annual)
        income_stmt = ticker.financials
        
        # Check if data is available
        if income_stmt.empty:
            print(f"No income statement data available for {ticker_symbol}.")
            return 'N/A'
        
        # Extract revenue data
        revenue_data = income_stmt.loc['Total Revenue']
        
        # Ensure the data is sorted by date (columns)
        revenue_data = revenue_data.sort_index(ascending=False)
        
        # Check if there are enough data points
        if len(revenue_data) < 2:
            print(f"Not enough revenue data to calculate growth rate for {ticker_symbol}.")
            return 'N/A'
        
        # Get the last two periods
        last_period_revenue = revenue_data.iloc[0]
        previous_period_revenue = revenue_data.iloc[1]
        
        # Calculate revenue growth rate
        growth_rate = (last_period_revenue - previous_period_revenue) / previous_period_revenue * 100
        
        return growth_rate
    
    except Exception as e:
        print(f"Error calculating growth rate for {ticker_symbol}: {e}")
        return 'N/A'
    
def get_revenue_growth_rate(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        income_stmt = ticker.financials
        
        if income_stmt.empty:
            print(f"No income statement data available for {ticker_symbol}.")
            return 'N/A'
        
        revenue_data = income_stmt.loc['Total Revenue']
        revenue_data = revenue_data.sort_index(ascending=False)
        
        if len(revenue_data) < 2:
            print(f"Not enough revenue data to calculate growth rate for {ticker_symbol}.")
            return 'N/A'
        
        last_period_revenue = safe_numeric_conversion(revenue_data.iloc[0])
        previous_period_revenue = safe_numeric_conversion(revenue_data.iloc[1])
        
        if last_period_revenue is None or previous_period_revenue is None:
            return 'N/A'
        
        growth_rate = (last_period_revenue - previous_period_revenue) / previous_period_revenue * 100
        return growth_rate
    
    except Exception as e:
        print(f"Error calculating growth rate for {ticker_symbol}: {e}")
        return 'N/A'


# Function to fetch stock data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    return stock.history(period="max")

def get_market_cap(ticker):
    """
    Fetch the market capitalization of a company using yfinance.

    Parameters:
    - ticker (str): The ticker symbol of the company (e.g., 'AAPL' for Apple Inc.)

    Returns:
    - market_cap (float): The market capitalization of the company in USD.
    """
    try:
        # Create a Ticker object
        stock = yf.Ticker(ticker)
        
        # Fetch the market capitalization from the info dictionary
        market_cap = stock.info.get('marketCap', None)
        
        # Check if market capitalization data is available
        if market_cap is None:
            raise ValueError(f"Market capitalization data not available for {ticker}")
        
        return market_cap
    except Exception as e:
        print(f"Error fetching market cap for {ticker}: {e}")
        return None

# Function to calculate ARR and MRR
#def calculate_recurring_revenue(income_statement):
    revenue = income_statement.loc['Total Revenue'].iloc[0]
    # Assuming all revenue is recurring for simplicity
    arr = revenue
    mrr = arr / 12
    return arr, mrr

# Function to calculate Burn Rate
#def calculate_burn_rate(financials):
    # Extract necessary data from financials
    total_expenses_annual = financials.loc['Total Expenses'].iloc[0]
    total_revenue_annual = financials.loc['Total Revenue'].iloc[0]
    
    # Convert annual values to monthly values
    total_expenses_monthly = total_expenses_annual / 12
    total_revenue_monthly = total_revenue_annual / 12
    
    # Calculate Gross Burn Rate
    gross_burn_rate = total_expenses_monthly
    
    # Calculate Net Burn Rate
    net_burn_rate = total_expenses_monthly - total_revenue_monthly
    
    return gross_burn_rate, net_burn_rate
def calculate_recurring_revenue(income_statement):
    try:
        revenue = safe_numeric_conversion(income_statement.loc['Total Revenue'].iloc[0])
        if revenue is None:
            return 'N/A', 'N/A'
        
        # Assuming all revenue is recurring for simplicity
        arr = revenue
        mrr = arr / 12
        return arr, mrr
    except Exception as e:
        print(f"Error calculating recurring revenue: {e}")
        return 'N/A', 'N/A'
def calculate_burn_rate(financials):
    try:
        total_expenses_annual = safe_numeric_conversion(financials.loc['Total Expenses'].iloc[0])
        total_revenue_annual = safe_numeric_conversion(financials.loc['Total Revenue'].iloc[0])
        
        if total_expenses_annual is None or total_revenue_annual is None:
            return 'N/A', 'N/A'
        
        total_expenses_monthly = total_expenses_annual / 12
        total_revenue_monthly = total_revenue_annual / 12
        
        gross_burn_rate = total_expenses_monthly
        net_burn_rate = total_expenses_monthly - total_revenue_monthly
        
        return gross_burn_rate, net_burn_rate
    except Exception as e:
        print(f"Error calculating burn rate: {e}")
        return 'N/A', 'N/A'

# Function to calculate Sales Efficiency
#def calculate_sales_efficiency(financials):
    # Get Total Revenue and Selling General And Administration
    total_revenue = financials.loc['Total Revenue'].iloc[0]
    selling_general_and_admin = financials.loc['Selling General And Administration'].iloc[0]
    
    # Calculate Sales Efficiency
    sales_efficiency = total_revenue / selling_general_and_admin
    return sales_efficiency

#def get_growth_rate(financials):
    # Calculate year-over-year growth rate using Total Revenue
    total_revenue = financials.loc['Total Revenue']
    growth_rate = ((total_revenue.iloc[0] - total_revenue.iloc[1]) / total_revenue.iloc[1]) * 100
    return growth_rate

#def get_profit_margin(financials):
    # Calculate profit margin using Net Income and Total Revenue
    net_income = financials.loc['Net Income'].iloc[0]
    total_revenue = financials.loc['Total Revenue'].iloc[0]
    profit_margin = (net_income / total_revenue) * 100
    return profit_margin
def calculate_sales_efficiency(financials):
    try:
        total_revenue = safe_numeric_conversion(financials.loc['Total Revenue'].iloc[0])
        selling_general_and_admin = safe_numeric_conversion(financials.loc['Selling General And Administration'].iloc[0])
        
        if total_revenue is None or selling_general_and_admin is None or selling_general_and_admin == 0:
            return 'N/A'
        
        sales_efficiency = total_revenue / selling_general_and_admin
        return sales_efficiency
    except Exception as e:
        print(f"Error calculating sales efficiency: {e}")
        return 'N/A'
def get_growth_rate(financials):
    try:
        total_revenue = financials.loc['Total Revenue']
        if len(total_revenue) < 2:
            return 'N/A'
        
        current_revenue = safe_numeric_conversion(total_revenue.iloc[0])
        previous_revenue = safe_numeric_conversion(total_revenue.iloc[1])
        
        if current_revenue is None or previous_revenue is None or previous_revenue == 0:
            return 'N/A'
        
        growth_rate = ((current_revenue - previous_revenue) / previous_revenue) * 100
        return growth_rate
    except Exception as e:
        print(f"Error calculating growth rate: {e}")
        return 'N/A'
    
def get_profit_margin(financials):
    try:
        net_income = safe_numeric_conversion(financials.loc['Net Income'].iloc[0])
        total_revenue = safe_numeric_conversion(financials.loc['Total Revenue'].iloc[0])
        
        if net_income is None or total_revenue is None or total_revenue == 0:
            return 'N/A'
        
        profit_margin = (net_income / total_revenue) * 100
        return profit_margin
    except Exception as e:
        print(f"Error calculating profit margin: {e}")
        return 'N/A'

def calculate_market_return(ticker, start_date, end_date):
    """
    Calculate the historical market return over a specified period.

    Parameters:
    ticker (str): The stock index ticker symbol (e.g., "^GSPC" for S&P 500).
    start_date (str): The start date for historical data (format: 'YYYY-MM-DD').
    end_date (str): The end date for historical data (format: 'YYYY-MM-DD').

    Returns:
    float: The annualized market return as a percentage.
    """
    # Retrieve historical data
    index = yf.Ticker(ticker)
    data = index.history(start=start_date, end=end_date)
    
    # Calculate the total return
    total_return = (data['Close'][-1] / data['Close'][0]) - 1
    
    # Annualize the return (assuming trading days are about 252 per year)
    num_years = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365
    annualized_return = (1 + total_return) ** (1 / num_years) - 1
    
    return annualized_return * 100  # Convert to percentage

#def passes_rule_of_40(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials

    if 'Total Revenue' not in financials.index or 'Net Income' not in financials.index:
        return None

    growth_rate = get_growth_rate(financials)
    profit_margin = get_profit_margin(financials)
    rule_of_40_value = growth_rate + profit_margin

    return rule_of_40_value
def passes_rule_of_40(ticker):
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials

        if 'Total Revenue' not in financials.index or 'Net Income' not in financials.index:
            return 'N/A'

        growth_rate = get_growth_rate(financials)
        profit_margin = get_profit_margin(financials)
        
        if growth_rate == 'N/A' or profit_margin == 'N/A':
            return 'N/A'
        
        rule_of_40_value = growth_rate + profit_margin
        return rule_of_40_value
    except Exception as e:
        print(f"Error calculating Rule of 40 for {ticker}: {e}")
        return 'N/A'
#net debt
#def calculate_net_debt(balance_sheet):
    """
    Calculate Net Debt using available balance sheet data.
    
    Parameters:
    - balance_sheet (DataFrame): The balance sheet DataFrame for the company.

    Returns:
    - net_debt (float or str): The net debt amount or 'N/A' if data is missing or cannot be computed.
    """
    try:
        # Retrieve data with safe default values if not available
        short_term_debt = balance_sheet.loc['Short Term Debt'].values[0] if 'Short Term Debt' in balance_sheet.index else 0
        long_term_debt = balance_sheet.loc['Long Term Debt'].values[0] if 'Long Term Debt' in balance_sheet.index else 0
        cash_and_cash_eq = balance_sheet.loc['Cash And Cash Equivalents'].values[0] if 'Cash And Cash Equivalents' in balance_sheet.index else 0

        # If total liabilities are present, calculate using them as a fallback
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'].values[0]
            total_debt = short_term_debt + long_term_debt if (short_term_debt or long_term_debt) else total_liabilities
        else:
            total_debt = short_term_debt + long_term_debt

        # Calculate Net Debt
        net_debt = total_debt - cash_and_cash_eq
        
        return net_debt if total_debt > 0 else 'N/A'
    except KeyError as e:
        print(f"Missing data for Net Debt calculation: {e}")
        return 'N/A'
    except Exception as e:
        print(f"Error calculating net debt: {e}")
        return 'N/A'
def calculate_net_debt(balance_sheet):
    try:
        short_term_debt = safe_numeric_conversion(balance_sheet.loc['Short Term Debt'].values[0]) if 'Short Term Debt' in balance_sheet.index else 0
        long_term_debt = safe_numeric_conversion(balance_sheet.loc['Long Term Debt'].values[0]) if 'Long Term Debt' in balance_sheet.index else 0
        cash_and_cash_eq = safe_numeric_conversion(balance_sheet.loc['Cash And Cash Equivalents'].values[0]) if 'Cash And Cash Equivalents' in balance_sheet.index else 0

        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liabilities = safe_numeric_conversion(balance_sheet.loc['Total Liabilities Net Minority Interest'].values[0])
            total_debt = short_term_debt + long_term_debt if (short_term_debt or long_term_debt) else total_liabilities
        else:
            total_debt = short_term_debt + long_term_debt

        if total_debt is None or cash_and_cash_eq is None:
            return 'N/A'

        net_debt = total_debt - cash_and_cash_eq
        return net_debt if total_debt > 0 else 'N/A'
    except Exception as e:
        print(f"Error calculating net debt: {e}")
        return 'N/A'
def calculate_total_debt(balance_sheet):
    try:

        total_debt = balance_sheet.loc['Total Debt']
        if isinstance(total_debt, pd.Series):
            return total_debt.values[0]  # Extract the single value from the Series
        return total_debt
    except Exception as e:
        print(f"Error retrieving cash and cash equivalents: {e}")
        return 'N/A'

def get_cce(balance_sheet):
    try:
        # Ensure that cash and cash equivalents is a single value
        cce = balance_sheet.loc['Cash And Cash Equivalents']
        if isinstance(cce, pd.Series):
            return cce.values[0]  # Extract the single value from the Series
        return cce
    except Exception as e:
        print(f"Error retrieving cash and cash equivalents: {e}")
        return 'N/A'



#def get_risk_free_rate(ticker='^IRX'):
    """
    Get the risk-free rate.

    Parameters:
    - ticker: Ticker symbol for the 13-week Treasury bill (default is '^IRX')

    Returns:
    - risk_free_rate: The annualized risk-free rate
    """

    # Fetch the 13-week Treasury bill yield as a proxy for the risk-free rate
    try:
        treasury_data = yf.Ticker(ticker).history(period="1d")
        risk_free_rate = treasury_data['Close'].iloc[-1] / 100  # Convert to decimal
        return risk_free_rate
    except Exception as e:
        print(f"Error fetching risk-free rate data: {e}")
        return None
def get_risk_free_rate(ticker='^IRX'):
    try:
        treasury_data = yf.Ticker(ticker).history(period="1d")
        risk_free_rate = safe_numeric_conversion(treasury_data['Close'].iloc[-1]) / 100  # Convert to decimal
        return risk_free_rate if risk_free_rate is not None else 'N/A'
    except Exception as e:
        print(f"Error fetching risk-free rate data: {e}")
        return 'N/A'

#def get_stock_beta(ticker):
    """
    Retrieve the stock's beta.
    """
    stock = yf.Ticker(ticker)
    beta = stock.info.get('beta', None)
    return beta
def get_stock_beta(ticker):
    try:
        stock = yf.Ticker(ticker)
        beta = safe_numeric_conversion(stock.info.get('beta', None))
        return beta if beta is not None else 'N/A'
    except Exception as e:
        print(f"Error fetching stock beta for {ticker}: {e}")
        return 'N/A'


#def calculate_required_rate_of_return(real_risk_free_rate, market_return, beta):
    """
    Calculate the required rate of return using CAPM.
    """
    required_rate = real_risk_free_rate + beta * (market_return - real_risk_free_rate)
    return required_rate
def calculate_required_rate_of_return(real_risk_free_rate, market_return, beta):
    try:
        real_risk_free_rate = safe_numeric_conversion(real_risk_free_rate)
        market_return = safe_numeric_conversion(market_return)
        beta = safe_numeric_conversion(beta)
        
        if real_risk_free_rate is None or market_return is None or beta is None:
            return 'N/A'
        
        required_rate = real_risk_free_rate + beta * (market_return - real_risk_free_rate)
        return required_rate
    except Exception as e:
        print(f"Error calculating required rate of return: {e}")
        return 'N/A'

#def get_dividend_per_share(ticker):
    """
    Retrieve the dividend per share for the preferred stock.
    
    Parameters:
    - ticker: The stock ticker symbol.
    
    Returns:
    - dividend_per_share: The dividend per share or 'N/A' if not available.
    """
    stock = yf.Ticker(ticker)
    dividends = stock.dividends
    
    if dividends.empty:
        print(f"No dividend data available for {ticker}. Company doesn't give out dividends.")
        return 'N/A'
    
    dividend_per_share = dividends.iloc[-1]  # Get the most recent dividend per share

    if pd.isna(dividend_per_share):
        print(f"Dividend per Share data is NaN for {ticker}.")
        return 'N/A'
    
    return dividend_per_share

# def calculate_dividend_yield(ticker):
    stock = yf.Ticker(ticker)

    # Get historical market data
    hist = stock.history(period="1d")

    # Get the most recent closing price
    price_per_share = hist['Close'].iloc[-1]

    # Fetch dividend data
    dividends = stock.dividends
    annual_dividends = dividends.resample('Y').sum().iloc[-1]  # Sum of annual dividends

    # Calculate Dividend Yield
    dividend_yield = annual_dividends / price_per_share
    return dividend_yield
    


# def calculate_preferred_stock_value(dividend_per_share, required_rate_of_return):
    """
    Calculate the value of preferred stock using the formula:
    Value = Dividend per Share / Required Rate of Return
    """
    
    if required_rate_of_return == 0:
        print(f"Cannot calculate preferred stock value: Required Rate of Return is zero.")
        return 'N/A'
    
    try:
        preferred_stock_value = dividend_per_share / required_rate_of_return
        return preferred_stock_value

    except Exception as e:
        print(f"Error calculating preferred stock value: {e}")
        return 'N/A'
def get_dividend_per_share(ticker):
    try:
        stock = yf.Ticker(ticker)
        dividends = stock.dividends
        
        if dividends.empty:
            print(f"No dividend data available for {ticker}. Company doesn't give out dividends.")
            return 'N/A'
        
        dividend_per_share = safe_numeric_conversion(dividends.iloc[-1])  # Get the most recent dividend per share
        return dividend_per_share if dividend_per_share is not None else 'N/A'
    except Exception as e:
        print(f"Error fetching dividend per share for {ticker}: {e}")
        return 'N/A'
def calculate_preferred_stock_value(dividend_per_share, required_rate_of_return):
    """
    Calculate the value of preferred stock using the formula:
    Value = Dividend per Share / Required Rate of Return
    """
    try:
        # Ensure inputs are numeric
        if isinstance(dividend_per_share, str) or isinstance(required_rate_of_return, str):
            if dividend_per_share == 'N/A' or required_rate_of_return == 'N/A':
                print(f"Cannot calculate preferred stock value: One or more inputs are 'N/A'.")
                return 'N/A'
            else:
                # Convert to float
                dividend_per_share = float(dividend_per_share)
                required_rate_of_return = float(required_rate_of_return)
        
        if required_rate_of_return == 0:
            print(f"Cannot calculate preferred stock value: Required Rate of Return is zero.")
            return 'N/A'
        
        preferred_stock_value = dividend_per_share / required_rate_of_return
        return preferred_stock_value

    except Exception as e:
        print(f"Error calculating preferred stock value: {e}")
        return 'N/A'


    
#def get_debt_to_equity_ratio(ticker):
    # Fetch the stock data
    stock = yf.Ticker(ticker)
    
    # Fetch the balance sheet data
    balance_sheet = stock.balance_sheet
    
    # Extract total liabilities and total equity
    total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
    total_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
    
    # Calculate the Debt to Equity Ratio
    debt_to_equity_ratio = total_liabilities / total_equity
    
    return debt_to_equity_ratio
def get_debt_to_equity_ratio(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet
        
        total_liabilities = safe_numeric_conversion(balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0])
        total_equity = safe_numeric_conversion(balance_sheet.loc['Stockholders Equity'].iloc[0])
        
        if total_liabilities is None or total_equity is None or total_equity == 0:
            return 'N/A'
        
        debt_to_equity_ratio = total_liabilities / total_equity
        return debt_to_equity_ratio
    except Exception as e:
        print(f"Error calculating debt to equity ratio for {ticker}: {e}")
        return 'N/A'

#def get_roe(ticker):
    # Fetch the stock data
    stock = yf.Ticker(ticker)
    
    # Fetch the financials and balance sheet data
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    
    # Extract net income and shareholder's equity
    net_income = financials.loc['Net Income'].iloc[0]
    total_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
    
    # Calculate ROE
    roe = net_income / total_equity
    
    return roe
def get_roe(ticker):
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        
        net_income = safe_numeric_conversion(financials.loc['Net Income'].iloc[0])
        total_equity = safe_numeric_conversion(balance_sheet.loc['Stockholders Equity'].iloc[0])
        
        if net_income is None or total_equity is None or total_equity == 0:
            return 'N/A'
        
        roe = net_income / total_equity
        return roe
    except Exception as e:
        print(f"Error calculating ROE for {ticker}: {e}")
        return 'N/A'
#def calculate_ev_revenue_multiple(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    return calculate_enterprise_value(ticker) / financials.loc['Total Revenue'].iloc[0]



#def calculate_enterprise_value(ticker):
    try:
        # Get financial data
        balance_sheet = get_balance_sheet_data(ticker)
        market_cap = get_market_cap(ticker)
        
        if market_cap is None:
            raise ValueError(f"Market capitalization data not available for {ticker}")
        
        # Calculate risk-free rate
        risk_free_rate = get_risk_free_rate(ticker='^IRX')
        if risk_free_rate is None:
            raise ValueError("Risk-free rate data not available.")
        
        # Calculate market return
        market_return = calculate_market_return('^GSPC', '2023-01-01', datetime.now().strftime('%Y-%m-%d'))
        
        # Get stock beta
        beta = get_stock_beta(ticker)
        if beta is None:
            raise ValueError("Stock beta data not available.")
        
        # Calculate required rate of return
        required_rate_of_return = calculate_required_rate_of_return(risk_free_rate, market_return, beta)
        
        # Calculate dividend per share
        dividend_per_share = get_dividend_per_share(ticker)
        if dividend_per_share is None:
            raise ValueError("Dividend per share data not available.")
        
        # Calculate preferred stock value
        preferred_stock_value = calculate_preferred_stock_value(dividend_per_share, required_rate_of_return)
        
        if preferred_stock_value == 'N/A':
            # Handle preferred stock value being 'N/A'
            preferred_stock_value = 0
        
        # Calculate Net Debt
        net_debt = calculate_net_debt(balance_sheet)
        if net_debt == 'N/A':
            # If net debt is 'N/A', set enterprise value to 'N/A' as well
            return 'N/A'
        
        #calculate total debt for more generalized formula since most of SaaS companies do not hand out preferred stock dividends :/
        total_debt = calculate_total_debt(balance_sheet)
        if total_debt == 'N/A':
            return 'N/A'
        
        # Calculate cash and cash equivalents
        cash_and_cash_eq = get_cce(balance_sheet)
        if cash_and_cash_eq == 'N/A':
            cash_and_cash_eq = 0
        
        # Calculate Enterprise Value
        enterprise_value = market_cap + total_debt - cash_and_cash_eq #not using preferred stock for now since this is more general formula
        
        return enterprise_value
    except Exception as e:
        print(f"Error calculating enterprise value for {ticker}: {e}")
        return 'N/A'
def calculate_ev_revenue_multiple(ticker):
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials
        ev = calculate_enterprise_value(ticker)
        revenue = safe_numeric_conversion(financials.loc['Total Revenue'].iloc[0])
        
        if ev == 'N/A' or revenue is None or revenue == 0:
            return 'N/A'
        
        return ev / revenue
    except Exception as e:
        print(f"Error calculating EV/Revenue multiple for {ticker}: {e}")
        return 'N/A'
def calculate_enterprise_value(ticker):
    try:
        balance_sheet = get_balance_sheet_data(ticker)
        market_cap = get_market_cap(ticker)
        
        if market_cap is None:
            return 'N/A'
        
        total_debt = calculate_total_debt(balance_sheet)
        cash_and_cash_eq = get_cce(balance_sheet)
        
        if total_debt == 'N/A' or cash_and_cash_eq == 'N/A':
            return 'N/A'
        
        enterprise_value = market_cap + total_debt - cash_and_cash_eq
        return enterprise_value
    except Exception as e:
        print(f"Error calculating enterprise value for {ticker}: {e}")
        return 'N/A'

#def get_ebitda(ticker):
    """
    Retrieve EBITDA for a given stock ticker.
    
    Parameters:
    - ticker: The stock ticker symbol.
    
    Returns:
    - ebitda: EBITDA value or 'N/A' if not available.
    """
    stock = yf.Ticker(ticker)
    
    # Fetch financial statements
    income_statement = stock.financials
    balance_sheet = stock.balance_sheet

    # Check if income statement contains EBITDA data
    if 'EBITDA' in income_statement.index:
        ebitda = income_statement.loc['EBITDA'].iloc[0]
        return ebitda
    else:
        print(f"EBITDA not directly available for {ticker}.")
        return 'N/A'

#def calculate_ebitda_multiple(ticker):
    ebitda = get_ebitda(ticker)
    ev = calculate_enterprise_value(ticker)
    return ev / ebitda
def get_ebitda(ticker):
    try:
        stock = yf.Ticker(ticker)
        income_statement = stock.financials

        if 'EBITDA' in income_statement.index:
            ebitda = safe_numeric_conversion(income_statement.loc['EBITDA'].iloc[0])
            return ebitda if ebitda is not None else 'N/A'
        else:
            print(f"EBITDA not directly available for {ticker}.")
            return 'N/A'
    except Exception as e:
        print(f"Error fetching EBITDA for {ticker}: {e}")
        return 'N/A'
def calculate_ebitda_multiple(ticker):
    try:
        ebitda = get_ebitda(ticker)
        ev = calculate_enterprise_value(ticker)
        
        if ebitda == 'N/A' or ev == 'N/A':
            return 'N/A'
        
        ebitda = safe_numeric_conversion(ebitda)
        ev = safe_numeric_conversion(ev)
        
        if ebitda is None or ev is None or ebitda == 0:
            return 'N/A'
        
        return ev / ebitda
    except Exception as e:
        print(f"Error calculating EBITDA multiple for {ticker}: {e}")
        return 'N/A'


# Function to implement retry logic for fetching stock data
def fetch_stock_data_with_retry(ticker, retries=3, delay=10):
    for i in range(retries):
        try:
            stock_data = fetch_stock_data(ticker)   
            return stock_data
        except Exception as e:
            print(f"Attempt {i+1} failed: {e}")
            time.sleep(delay)
    raise Exception(f"Failed to fetch stock data for {ticker} after {retries} attempts")

# Function to read tickers from CSV and calculate metrics
#def process_tickers(csv_path, output_path):

    # Read tickers from CSV
    tickers = pd.read_csv(csv_path)['Ticker']
    
    # Initialize a DataFrame to store results
    results = []
    
    for ticker in tickers:
        try:
            # Fetch financial data
            income_statement = get_income_statement_data(ticker)
            financial_data = get_financial_data(ticker)
            cash_flow_data = get_cash_flow_data(ticker)
            balance_sheet_data = get_balance_sheet_data(ticker)
            
            # Calculate metrics
            gross_margin = calculate_gross_margin(income_statement)
            revenue_growth_rate = get_revenue_growth_rate(ticker)
            arr, mrr = calculate_recurring_revenue(income_statement)
            gross_burn_rate, net_burn_rate = calculate_burn_rate(financial_data)
            sales_efficiency = calculate_sales_efficiency(financial_data)
            profit_margin = get_profit_margin(financial_data)
            growth_rate = get_growth_rate(financial_data)
            rule_40 = passes_rule_of_40(ticker)
            #net_debt = calculate_net_debt(balance_sheet_data)
            total_debt = calculate_total_debt(balance_sheet_data)
            beta = get_stock_beta(ticker)
            EBITDA_multiple = calculate_ebitda_multiple(ticker)
            #dividend_yield = calculate_dividend_yield(ticker)
            ev_revenue_mult = calculate_ev_revenue_multiple(ticker)
            deratio = get_debt_to_equity_ratio(ticker)
            roe = get_roe(ticker)
            
            # Calculate Preferred Stock Value
            dividend_per_share = get_dividend_per_share(ticker)
            print(f"Dividend per Share for {ticker}: {dividend_per_share}")  # Debugging line
            
            risk_free_rate = get_risk_free_rate(ticker='^IRX')
            market_return = calculate_market_return('^GSPC', '2023-01-01', datetime.now().strftime('%Y-%m-%d'))
            required_rate_of_return = calculate_required_rate_of_return(risk_free_rate, market_return, beta)
            print(f"Required Rate of Return for {ticker}: {required_rate_of_return}")  # Debugging line
            
            # Ensure preferred stock value calculation
            if dividend_per_share is not None and required_rate_of_return is not None and required_rate_of_return != 0:
                preferred_stock_value = calculate_preferred_stock_value(dividend_per_share, required_rate_of_return)
            else:
                preferred_stock_value = 'N/A'
            
            # Append results
            result = {
                'Ticker': ticker,
                'Profit Margin': profit_margin,
                'Growth Rate': growth_rate,
                'Rule of 40 (T/F)': rule_40,
                'Gross Margin': gross_margin,
                'Revenue Growth Rate': revenue_growth_rate,
                'Annual Recurring Revenue (ARR)': arr,
                'Monthly Recurring Revenue (MRR)': mrr,
                'Gross Burn Rate': gross_burn_rate,
                'Net Burn Rate': net_burn_rate,
                'Sales Efficiency': sales_efficiency,
                'Total Debt': total_debt,
                'Beta': beta,
                'Enterprise Value': calculate_enterprise_value(ticker),
                'EBITDA Multiple' : EBITDA_multiple,
                'Preferred Stock Value': preferred_stock_value,
                'Enterprise Value/Revenue' : ev_revenue_mult,
                'Debt to Equity Ratio' : deratio,
                'Return on Equity' : roe

            }
            results.append(result)
        except Exception as e:
            print(f"Failed to process {ticker}: {e}")
    
    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)

def process_tickers(csv_path, output_path):
    tickers = pd.read_csv(csv_path)['Ticker']
    results = []
    
    for ticker in tickers:
        try:
            # Fetch data
            income_statement = get_income_statement_data(ticker)
            financial_data = get_financial_data(ticker)
            cash_flow_data = get_cash_flow_data(ticker)
            balance_sheet_data = get_balance_sheet_data(ticker)
            
            # Calculate metrics
            gross_margin = calculate_gross_margin(income_statement)
            revenue_growth_rate = get_revenue_growth_rate(ticker)
            arr, mrr = calculate_recurring_revenue(income_statement)
            gross_burn_rate, net_burn_rate = calculate_burn_rate(financial_data)
            sales_efficiency = calculate_sales_efficiency(financial_data)
            profit_margin = get_profit_margin(financial_data)
            growth_rate = get_growth_rate(financial_data)
            rule_40 = passes_rule_of_40(ticker)
            total_debt = calculate_total_debt(balance_sheet_data)
            beta = get_stock_beta(ticker)
            EBITDA_multiple = calculate_ebitda_multiple(ticker)
            ev_revenue_mult = calculate_ev_revenue_multiple(ticker)
            deratio = get_debt_to_equity_ratio(ticker)
            roe = get_roe(ticker)
            
            # Calculate Preferred Stock Value
            dividend_per_share = get_dividend_per_share(ticker)
            print(f"Dividend per Share for {ticker}: {dividend_per_share}")  # Debugging line
            
            risk_free_rate = get_risk_free_rate(ticker='^IRX')
            market_return = calculate_market_return('^GSPC', '2023-01-01', datetime.now().strftime('%Y-%m-%d'))
            required_rate_of_return = calculate_required_rate_of_return(risk_free_rate, market_return, beta)
            print(f"Required Rate of Return for {ticker}: {required_rate_of_return}")  # Debugging line
            
            if dividend_per_share != 'N/A' and required_rate_of_return not in [None, 'N/A', 0]:
                preferred_stock_value = calculate_preferred_stock_value(dividend_per_share, required_rate_of_return)
            else:
                preferred_stock_value = 'N/A'
            
            # Append results
            result = {
                'Ticker': ticker,
                'Profit Margin': profit_margin,
                'Growth Rate': growth_rate,
                'Rule of 40 (T/F)': rule_40,
                'Gross Margin': gross_margin,
                'Revenue Growth Rate': revenue_growth_rate,
                'Annual Recurring Revenue (ARR)': arr,
                'Monthly Recurring Revenue (MRR)': mrr,
                'Gross Burn Rate': gross_burn_rate,
                'Net Burn Rate': net_burn_rate,
                'Sales Efficiency': sales_efficiency,
                'Total Debt': total_debt,
                'Beta': beta,
                'Enterprise Value': calculate_enterprise_value(ticker),
                'EBITDA Multiple': EBITDA_multiple,
                'Preferred Stock Value': preferred_stock_value,
                'Enterprise Value/Revenue': ev_revenue_mult,
                'Debt to Equity Ratio': deratio,
                'Return on Equity': roe
            }
            results.append(result)
        except Exception as e:
            print(f"Failed to process {ticker}: {e}")
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)



# Example usage
input_csv = 'AllDatav2.csv'
output_csv = 'valuation_metrics10.csv'
process_tickers(input_csv, output_csv)


All these metrics can be found here: https://docs.google.com/spreadsheets/d/1SEIanQtt9br6DuvXBDwuDBxG6fgJOGQDPQ9UH_P7HAU/edit?usp=sharing 

The most important valuation factors to note is the EBITDA multiple, Debt to Equity Ratio, and finally the enterprise value, a comprehensive number that takes into account the market cap and debt levels of any company. 